In [ ]:
# Fold
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 256, 256
train_data_dir = "data/train"
validation_data_dir = "data/val"
nb_train_samples = 4125
nb_validation_samples = 466 
batch_size = 16
epochs = 50

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 1))

"""
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_conv3 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_conv4 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 32, 32, 256)       0         
_________________________________________________________________
block4_conv1 (Conv2D)        (None, 32, 32, 512)       1180160   
_________________________________________________________________
block4_conv2 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_conv3 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_conv4 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_pool (MaxPooling2D)   (None, 16, 16, 512)       0         
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv4 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 8, 8, 512)         0         
=================================================================
Total params: 20,024,384.0
Trainable params: 20,024,384.0
Non-trainable params: 0.0
"""

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(16, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])



In [10]:
# fold
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# add magic
from keras import backend as K
K.set_image_dim_ordering('th')

def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    print('dataSpecs = ', dataSpecs)
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x
 
    
    
def main():
    # Get images and labels
    print('Started')
    data = {'img': 0, 'label': 0}
    dataSpecs = {'classLength': []}
    dataSpecs['classLength'] = []
    data['img'] = np.concatenate((
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/0/*', dataSpecs), # Class 0
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/1/*', dataSpecs) # Class 1
            get_images('/user/HS204/wm0015/student/allCalcs/0/*', dataSpecs), # Class 0
            get_images('/user/HS204/wm0015/student/allCalcs/1/*', dataSpecs) # Class 1
    ))  
    print('*** data[img].shape = ', data['img'].shape)
    for _ in range(385):
        print(data['img'][0,_,50,:]) 
    print('dataSpecs = ', dataSpecs['classLength'])
    print(dataSpecs['classLength'][1])
    labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
    labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
    data['label'] = np.concatenate((
            get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
            get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
    ))
    # Drop from 3 colour channels to 1 (greyscale)
    data['img'] = data['img'][:,:,:,0]
    data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
    print('new data shape = ', data['img'].shape)
    # Shuffle data
#     seed = 33
#     np.random.seed(seed) # Has to be set before each use of random
#     shuffleMask = np.random.permutation(data['img'].shape[0])    
#     data['img'] = data['img'][shuffleMask, :, :, :]
#     data['label'] = data['label'][shuffleMask, :]
    
    # Split traing and validation data        
    splitRatio = 0.9
    splitPoint = math.floor(data['img'].shape[0]*splitRatio)
    train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
    validation_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
    print('train_data[img].shape = ', train_data['img'].shape)
    print('train_data[label].shape = ', train_data['label'].shape)
    print('validation_data[img].shape = ', validation_data['img'].shape)
    print('valdiation_data[label].shape = ', validation_data['label'].shape)
    
    # Print image    
    img_calc = train_data['img']/255    
#     plt.imshow(img_calc[0], cmap='gray')
#     plt.show()
    
    # Keras!
    model = Sequential()
    
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(1,28,28))) # (385, 385,1)
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu'))   
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 5
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    
    from keras.datasets import mnist
    # Load pre-shuffled MNIST data into train and test sets
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
    X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
    print(X_train.shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    # Convert 1-dimensional class arrays to 10-dimensional class matrices
    Y_train = np_utils.to_categorical(y_train, 10)
    Y_test = np_utils.to_categorical(y_test, 10)
    print(Y_train.shape)
    
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy',
            optimizer=adam,
            metrics=['accuracy']) # Accuracy will now be returned when evaluate is called
    # Train
#     model.fit(train_data['img'], train_data['label'], 
#           batch_size=100, epochs=100, verbose=1)
    model.fit(X_train, Y_train, 
      batch_size=10, epochs=6, verbose=1)
    
    # Evaluate
    # I think by default this returns the loss and accuracy
    #score = model.evaluate(validation_data['img'], validation_data['label'], verbose=0)
    score = model.evaluate(X_train, Y_train, verbose=0)
    print('The score is......\n', score)
    
if __name__ == "__main__":
    main()

Started
dataSpecs =  {'classLength': [901]}
dataSpecs =  {'classLength': [901, 719]}
*** data[img].shape =  (1620, 385, 385, 3)
[64 64 64]
[64 64 64]
[61 61 61]
[53 53 53]
[47 47 47]
[43 43 43]
[40 40 40]
[36 36 36]
[32 32 32]
[29 29 29]
[30 30 30]
[32 32 32]
[32 32 32]
[36 36 36]
[42 42 42]
[44 44 44]
[44 44 44]
[49 49 49]
[55 55 55]
[58 58 58]
[59 59 59]
[58 58 58]
[56 56 56]
[58 58 58]
[61 61 61]
[62 62 62]
[60 60 60]
[59 59 59]
[60 60 60]
[62 62 62]
[61 61 61]
[57 57 57]
[53 53 53]
[49 49 49]
[44 44 44]
[46 46 46]
[53 53 53]
[57 57 57]
[58 58 58]
[60 60 60]
[61 61 61]
[62 62 62]
[69 69 69]
[78 78 78]
[82 82 82]
[84 84 84]
[89 89 89]
[94 94 94]
[100 100 100]
[106 106 106]
[109 109 109]
[106 106 106]
[101 101 101]
[96 96 96]
[86 86 86]
[77 77 77]
[73 73 73]
[73 73 73]
[72 72 72]
[71 71 71]
[75 75 75]
[86 86 86]
[95 95 95]
[100 100 100]
[103 103 103]
[109 109 109]
[122 122 122]
[134 134 134]
[136 136 136]
[130 130 130]
[125 125 125]
[121 121 121]
[116 116 116]
[113 113 113]
[107 107 1

In [3]:
# 3. Import libraries and modules
import numpy as np
from keras import backend as K
K.set_image_dim_ordering('th')
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
 
# 4. Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
# 5. Preprocess input data
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
print(Y_train.shape)
print(X_train.shape)
 
# 7. Define model architecture
model = Sequential()
 
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(1,28,28)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
# 9. Fit model on training data
model.fit(X_train, Y_train, 
          batch_size=32, nb_epoch=10, verbose=1)
 
# 10. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)

(60000, 10)
(60000, 1, 28, 28)


/user/HS204/wm0015/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(1, 28, 28...)`
/user/HS204/wm0015/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/user/HS204/wm0015/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
60000/60000 [==============================] - 12s 205us/step - loss: 0.2067 - acc: 0.9358
Epoch 2/10
60000/60000 [==============================] - 12s 193us/step - loss: 0.0864 - acc: 0.9746
Epoch 3/10
60000/60000 [==============================] - 11s 183us/step - loss: 0.0663 - acc: 0.9798
Epoch 4/10
60000/60000 [==============================] - 11s 175us/step - loss: 0.0554 - acc: 0.9828
Epoch 5/10
60000/60000 [==============================] - 11s 182us/step - loss: 0.0477 - acc: 0.9857
Epoch 6/10
60000/60000 [==============================] - 11s 178us/step - loss: 0.0436 - acc: 0.9865
Epoch 7/10
60000/60000 [==============================] - 10s 172us/step - loss: 0.0389 - acc: 0.9875
Epoch 8/10
60000/60000 [==============================] - 11s 187us/step - loss: 0.0354 - acc: 0.9889
Epoch 9/10
60000/60000 [==============================] - 11s 189us/step - loss: 0.0322 - acc: 0.9901
Epoch 10/10
60000/60000 [==============================] - 11s 189us/step - loss: 

In [2]:
a = np.array([np.arange(9) for i in range(3)])
a = a + 2
print(a)
print(a.shape)
print('__')
b = np.random.permutation(a.shape[1])
print(b)
print(a[1,b])
print('\n\n')
np.random.seed(10)  # Has to be set before each use of random
a = a[:, np.random.permutation(a.shape[1])]
print(a)


[[ 2  3  4  5  6  7  8  9 10]
 [ 2  3  4  5  6  7  8  9 10]
 [ 2  3  4  5  6  7  8  9 10]]
(3, 9)
__
[0 8 4 5 2 6 3 1 7]
[ 2 10  6  7  4  8  5  3  9]



[[10  4  7  8  5  3  2  9  6]
 [10  4  7  8  5  3  2  9  6]
 [10  4  7  8  5  3  2  9  6]]


In [15]:
print('hello world')

hello world


In [39]:
# Mutable objects such as lists act as call by reference
# Unmutable objects such as integers, strings act as call by value

def test (f):
    f[0] = 2
x = [1, 2, 3]
test(x)
print(x)

[2, 2, 3]


In [64]:

def myFunc (x):
    x = x+[99]
    print(x, '\n')
def alterDict(x):
    x['test'] = [9,9,9]
def alterList(x):
    x = x + [7]
    
myDict = {'test': [1,2,3]}
print(myDict)
myFunc(myDict['test'])

print(myDict)
alterDict(myDict)
print(myDict)

y = [1,2,3]
alterList(y)
print('\n',y)


{'test': [1, 2, 3]}
[1, 2, 3, 99] 

{'test': [1, 2, 3]}
{'test': [9, 9, 9]}

 [1, 2, 3]
